## Example script: extracting tau classifier for dentate nucleus regions

**Read in relevant files**

In [1]:
import sys
sys.path.insert(0,
                '/Users/mokur/OneDrive - University of Cambridge/Attachments/Jan2023/Tau_pipeline/Tau_classification/')

from base import *
from constants import *
from tau_classification import *
import joblib

### Load pre-trained models

**Screening classifier**

In [2]:
path = "/Users/mokur/OneDrive - University of Cambridge/Attachments/Jan2023/Tau_pipeline/Tau_classification/Pre-trained/Models/"
filename = "screening_classifier_updated3.sav"
screening_model = joblib.load(path+filename)

In [3]:
# Some checks
print(screening_model.pipeline)
screening_model.f_importance.head()

Pipeline(steps=[('normalizer', MinMaxScaler()),
                ('selector',
                 RFE(estimator=RandomForestClassifier(random_state=42),
                     n_features_to_select=46)),
                ('clf',
                 BalancedRandomForestClassifier(class_weight='balanced',
                                                max_features=1,
                                                min_samples_leaf=2,
                                                n_estimators=600,
                                                random_state=42))])


,features,importance
33,ROI: 0.25 µm per pixel: Hematoxylin: Mean,0.043437
34,ROI: 0.25 µm per pixel: Hematoxylin: Median,0.041947
37,ROI: 0.25 µm per pixel: Red: Mean,0.038536
36,ROI: 0.25 µm per pixel: Red: Max,0.036822
19,ROI: 0.25 µm per pixel: DAB: Haralick Sum aver...,0.034232


**Tau classifier for dentate nucleus**

In [4]:
path = "/Users/mokur/OneDrive - University of Cambridge/Attachments/Jan2023/Tau_pipeline/Tau_classification/Pre-trained/Models/"
filename = "dn_classifier.sav"
dn_model = joblib.load(path+filename)

In [5]:
print(dn_model.pipeline)
print(dn_model.best_parameters)
dn_model.f_importance.head()

Pipeline(steps=[('normalizer', MinMaxScaler()),
                ('selector',
                 RFE(estimator=RandomForestClassifier(random_state=42),
                     n_features_to_select=34)),
                ('clf',
                 BalancedRandomForestClassifier(class_weight='balanced',
                                                max_features=0.2,
                                                random_state=42,
                                                sampling_strategy='not '
                                                                  'majority'))])
{0: (0.409, 0.9144010858386309, 0.9031204850361197, 0.9314285714285713), 1: (0.422, 0.9811932262266609, 0.9714598662207358, 0.9916666666666668), 2: (0.6599999999999999, 0.9944717736676886, 0.9934360958126922, 0.9955770411295273)}


,features,importance
0,Area µm^2,0.216869
4,Min diameter µm,0.207962
3,Max diameter µm,0.058139
2,Length µm,0.056409
19,ROI: 0.25 µm per pixel: DAB: Haralick Sum entr...,0.040231


### Putting them together: tau classification pipeline for novel *dentate nucleus* slides

In [6]:
novel_path = '/Users/mokur/OneDrive - University of Cambridge/Attachments/Jan2023/Detections/DN/'
novel_filename = 'detections.txt'
prediction_path = "C:/Users/mokur/OneDrive/Desktop/Digital_path/Predictions/DN/"

**Reading in files**

In [7]:
with open(novel_path + novel_filename) as f:
    mylist = f.read().splitlines()

print("Read in: ", len(mylist), "files")

Read in:  32 files


#### Tau classification

In [8]:
n_total = len(mylist)
faulty_file = []
for i in range(0, n_total):

    # Read in novel slide
    print("FILE", mylist[i], "Number: ", i + 1, "/", n_total)
    print("---------------STEP1: Read in data file -------------------")
    dat_file = mylist[i]

    dat_ = pd.read_csv(novel_path + dat_file, sep="\t")

    # Fixing order of the columns
    ordered = dat_[extracted_features]

    # Changing column names
    # since these names tend to be inconsistent causing problems
    ordered.columns.values[5] = "Centroid_X"
    ordered.columns.values[6] = "Centroid_Y"

    dat = ordered[ordered["Class"] == "Unlabelled"]  # only unlabelled cells
    print("Read in data file:", dat_file)
    print("Data shape is:", dat.shape)

    # Classifier 1: separating Non-tau from Tau
    print(
        "---------------STEP2: Separating Non-tau from Tau -------------------"
    )
    # 1) Remove NA cells
    dat = dat.dropna()

    predicted1_slide = dat.copy()

    # 2) Dropping extra info features
    X_unlabelled = dat.drop(
        columns=["Image",
                 "Name",
                 "Class",
                 "Parent",
                 "ROI",
                 "Centroid_X",
                 "Centroid_Y"]
    )
    # 3) Predictions
    screening_model.predict(X_unlabelled)
    predicted1_slide["Class"] = screening_model.prediction
    print(predicted1_slide["Class"].value_counts())

    # Classifier 2: tau hallmark classification
    print(
        "---------------STEP3: Tau hallmark classification -------------------"
    )
    # Select out 'tau' portion only (ignoring non-tau & ambiguous cells )
    tau_portion = predicted1_slide[predicted1_slide["Class"] == "Tau"]
    if tau_portion.shape[0] == 0:
        print("There is no tau on this slide!")
        faulty_file.append(dat["Image"][0] + " No tau on the slide")
        continue
    tau_portion_X = tau_portion.drop(
        columns=["Image",
                 "Name",
                 "Class",
                 "Parent",
                 "ROI",
                 "Centroid_X",
                 "Centroid_Y"])
    predicted2_slide = tau_portion.copy()
    # Separates out detections that are not tau
    non_tau_portion = predicted1_slide[predicted1_slide["Class"] != "Tau"]

    # Make predictions on Tau objects
    dn_model.predict(tau_portion_X)
    predicted2_slide["Class"] = dn_model.prediction
    print(predicted2_slide["Class"].value_counts())

    # Extracting data out
    print("---------------STEP4: Data extraction & export -------------------")

    # 1) Combining predicted cells & excluded cells (prior to prediction)
    total_pred = pd.concat([non_tau_portion,
                            predicted2_slide])
    print("No loss of cells? ",
          predicted1_slide.shape[0] == total_pred.shape[0])

    output_visualise = total_pred[["Image",
                                   "Class",
                                   "Centroid_X",
                                   "Centroid_Y",
                                   "Area µm^2"]]
    path_ = (
            prediction_path +
            output_visualise.iloc[0, 0] +
            "_predictions.txt"
            )
    output_visualise.to_csv(path_, sep="\t", index=False)

    print("Exported prediction of : ", dat_file)
    print("---------------------------------------------------")
print("Well done, no error!")

FILE 703480.svs Detections.txt Number:  1 / 32
---------------STEP1: Read in data file -------------------
Read in data file: 703480.svs Detections.txt
Data shape is: (2808, 61)
---------------STEP2: Separating Non-tau from Tau -------------------
Non_tau    2787
Tau          21
Name: Class, dtype: int64
---------------STEP3: Tau hallmark classification -------------------
Others    20
NFT        1
Name: Class, dtype: int64
---------------STEP4: Data extraction & export -------------------
No loss of cells?  True
Exported prediction of :  703480.svs Detections.txt
---------------------------------------------------
FILE 703491.svs Detections.txt Number:  2 / 32
---------------STEP1: Read in data file -------------------
Read in data file: 703491.svs Detections.txt
Data shape is: (16744, 61)
---------------STEP2: Separating Non-tau from Tau -------------------
Non_tau    11186
Tau         5558
Name: Class, dtype: int64
---------------STEP3: Tau hallmark classification ------------------